<a href="https://colab.research.google.com/github/rsotop/Anal-tica-de-negocio/blob/main/BPMN_con_ChatGPT_Compartido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Configuración de entorno GPT
#### Cargue la clave API y las bibliotecas de Python relevantes.

In [ ]:
!pip install openai

import openai
import os
openai.api_key = 'sk-...'

#### Función auxiliar 
Utilizaremos el modelo gpt-3.5-turbo de OpenAI [Consulte guía](https://platform.openai.com/docs/guides/chat). 

Esta función auxiliar facilitará el uso de indicaciones y la visualización de los resultados generados:

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # este es el grado de aleatoriedad de la salida del modelo
    )
    return response.choices[0].message["content"]

#### Táctica: use delimitadores para indicar claramente distintas partes de la entrada
- Los delimitadores pueden ser cualquier cosa como: ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
Descripción del proceso:

A una empresa de hardware informático le gustaría modelar \
su proceso de envío para que sea visible y claro para \
todos en la empresa. Aquí hay una descripción del proceso \
según lo descrito por el gerente de la oficina:
Al enviar pedidos, primero determino el tamaño físico del pedido.
Si determino que el tamaño es más grande que una caja de envío regular\
(también conocida como "entrega especial"), me comunico con tres \
compañías de envío y le pido a cada una una una oferta. \
Luego selecciono la mejor oferta e informo al proveedor del envío pendiente.
Si, por otro lado, determino que el pedido puede usar una caja de envío regular, \
imprimo una etiqueta y verifico si es necesario un seguro.
Si el seguro es necesario, la gente de Logística se encargará \
de la adquisición del seguro de envío.
Mientras eso sucede, la gente en el almacén está reuniendo y \
empacando los artículos requeridos. 
Una vez que se ha determinado y configurado el método de envío, \
las personas en el almacén preparan los productos para ser recogidos \
por el proveedor del método de envío seleccionado.

Consejos:
- El alcance del proceso es solo preparar el envío, no el envío en sí.
- El proceso debe ser para la organización interna. \
¡Las compañías navieras no son parte de ella!

"""

prompt = f"""
Generar un objeto XML de proceso descrito que sea \
interpretable para la herramienta Camunda Modeler. \
El modelo debe estar basado en recomendaciones y mejores \
prácticas de modelado con BPMN, procurando el completo \ 
y correcto uso, etiquetado, distribución y conexión \
de elementos (eventos, tareas, compuertas, flujos, etc.).

\"\"\"{text}\"\"\"
"""
bpmnXML = get_completion(prompt)
print(bpmnXML)

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_1" targetNamespace="http://bpmn.io/schema/bpmn" exporter="Camunda Modeler" exporterVersion="4.4.0">
  <bpmn:process id="Process_1" name="Proceso de envío">
    <bpmn:startEvent id="StartEvent_1">
      <bpmn:outgoing>SequenceFlow_1</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:sequenceFlow id="SequenceFlow_1" sourceRef="StartEvent_1" targetRef="Task_1" />
    <bpmn:task id="Task_1" name="Determinar tamaño físico del pedido">
      <bpmn:incoming>SequenceFlow_1</bpmn:incoming>
      <bpmn:outgoing>SequenceFlow_2</bpmn:outgoing>
    </bpmn:task>
    <bpmn:exclusiveGateway id="ExclusiveGateway_1" name="¿Es una entrega especial?">
      <bpmn:incoming>S

### Configuración de entorno BPMN.io

In [ ]:
from IPython.display import HTML

def generate_bpmn_html(bpmn_xml):
    # Código HTML para cargar bpmn-js y mostrar el diagrama
    html_content = f"""
    <html>
        <head>
            <title>BPMN Viewer</title>
            <script src="https://cdn.jsdelivr.net/npm/bpmn-js/dist/bpmn-viewer.production.min.js"></script>
            <style>
                #canvas {{
                    height: 600px;
                    width: 100%;
                }}
            </style>
        </head>
        <body>
            <div id="canvas"></div>
            <script>
                const bpmnXML = `{bpmn_xml}`;
                const viewer = new BpmnJS({{ container: '#canvas' }});

                viewer.importXML(bpmnXML, function(err) {{
                    if (err) {{
                        console.error('Error rendering BPMN diagram', err);
                    }} else {{
                        console.log('BPMN diagram rendered.');
                    }}
                }});
            </script>
        </body>
    </html>
    """

    return html_content

html = generate_bpmn_html(bpmnXML)
HTML(html)

#### Notas sobre el uso de la API de OpenAI

Para instalar la biblioteca OpenAI Python:
```
!pip install openai
```

La biblioteca debe configurarse con la clave secreta de su cuenta, que está disponible en el [sitio web](https://platform.openai.com/account/api-keys). 

Puede configurarlo como la variable de entorno  `OPENAI_API_KEY` antes de usar la biblioteca:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

O bien, establezca `openai.api_key` en su valor:

```
import openai
openai.api_key = "sk-..."

#### Una nota sobre la barra invertida (backslash)
- Se usa una barra invertida \ para hacer que el texto quepa en la pantalla sin insertar caracteres de nueva línea '\n'.
- GPT-3 no se ve realmente afectado si inserta caracteres de nueva línea o no. Pero cuando trabaje con LLM en general, puede considerar si los caracteres de nueva línea en su solicitud pueden afectar el rendimiento del modelo.

<h4>Referencias: <a href="https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines">ChatGPT Prompt Engineering for Developers</a>, <a href="https://academy.camunda.com/">Camunda Academy</a>, <a href="https://https://bpmn.io//">BPMN.iO</a>.</h4>
